In [2]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import tensorflow as tf
import tensorflow_probability as tfp

sys.path.append(os.path.abspath(os.path.join('../BayesFlow')))

from bayesflow.trainers import ParameterEstimationTrainer
from bayesflow.networks import InvertibleNetwork, InvariantNetwork
from bayesflow.amortizers import SingleModelAmortizer
from bayesflow.models import GenerativeModel
from bayesflow.diagnostics import true_vs_estimated
from bayesflow.exceptions import ConfigurationError

from meta_aux_classes import *

In [33]:
class GaussianPrior:
    def __init__(self, D, mu_mean=0, mu_scale=1):
        self.D = D
        self.mu_mean = mu_mean
        self.mu_scale = mu_scale
    
    def __call__(self, n_sim):
        theta = np.random.default_rng().normal(self.mu_mean, self.mu_scale, size=(n_sim, self.D))
        return theta

class GaussianSimulator:
    def __init__(self, D, s = None):
        
        # Default: Unit variance
        if s is None:
            sigma = np.eye(D)
            
        # Unit variance with factor s
        elif isinstance(s, (int, float)):
            sigma = np.eye(D) * s
            
            
        # s is list or np.array --> Either custom diagonal or full
        elif isinstance(s, (list, np.ndarray)):
            # cast any list-like input to float np.array 
            if isinstance(s, list):
                s = np.array(s)   
            s = s.astype(float)
            
            # Diagonal covariance matrix with different diagonal entries (from s)
            if s.ndim == 1:
                assert len(s) == D, "Must provide D entries in diagonal s!"
                sigma = np.diag(s)
                
            # Full covariance matrix    
            elif s.ndim == 2:
                assert s.shape[0] == D and s.shape[1] == D, "Must provide DxD matrix!"
                try: 
                    _ = np.linalg.cholesky(s)
                except:
                    raise ConfigurationError("Covariance Matrix must be positive semidefinite!")
                
                sigma = s
            
        self.D = D    
        self.sigma = sigma
    
    def __call__(self, theta, n_obs):
        n_sim, D = theta.shape
        assert D == self.D
        
        tril = tf.linalg.cholesky(np.stack([self.sigma] * n_sim))   # tf requires cholesky decomposed sigma
        
        mvn = tfp.distributions.MultivariateNormalTriL(loc=theta, scale_tril=tril)

        sim_data = mvn.sample(n_obs)
        sim_data = np.array(sim_data)
        sim_data = np.transpose(sim_data, (1, 0, 2))
        return x
    
D = 3
prior = GaussianPrior(D=D)
simulator = GaussianSimulator(D=D)

In [34]:
D = 3
prior = GaussianPrior(D=D)
simulator = GaussianSimulator(D=D, s = 2.5)

theta = prior(4)
print(simulator.sigma)
simulator(theta, 5)

[[2.5 0.  0. ]
 [0.  2.5 0. ]
 [0.  0.  2.5]]


array([[[ 2.73411521,  1.2185937 ,  0.63388004],
        [ 0.32379717, -0.48083813, -1.06348095],
        [-1.55654599,  0.30787907, -1.00003755],
        [ 2.67841298, -0.86025733,  1.23173479],
        [ 1.48627776,  0.09162066,  0.79058084]],

       [[ 1.17545734, -2.40205846,  1.19263574],
        [ 3.3972544 ,  0.87160057,  0.40626682],
        [ 2.81696949, -2.47671772,  0.41413713],
        [ 2.67019141, -1.65938332, -2.55494686],
        [ 2.76689629,  1.1157164 ,  0.47626679]],

       [[ 3.00690881, -2.53231662, -1.62994585],
        [ 1.17783036, -3.63151215,  3.73122725],
        [ 2.43744483,  1.15886377,  2.36298164],
        [ 0.61906866, -2.64339331,  1.02858368],
        [ 1.19296863, -0.84260406,  3.40286183]],

       [[-1.43016473,  0.38395082, -1.04635111],
        [ 0.65995847, -0.94024824,  0.07733434],
        [ 4.03576003, -1.43655964,  0.79615501],
        [-0.14090567,  0.10978985,  3.33887821],
        [ 0.91678204, -0.57093465, -0.65044482]]])

In [35]:
D = 3
prior = GaussianPrior(D=D)
simulator = GaussianSimulator(D=D, s = [1,2,3])

theta = prior(4)
print(simulator.sigma)
simulator(theta, 5)

[[1. 0. 0.]
 [0. 2. 0.]
 [0. 0. 3.]]


array([[[ 1.36365241, -0.26539497,  0.72592553],
        [ 0.97858719, -1.45104664,  1.45481625],
        [ 1.83823963, -2.32313304,  1.26984988],
        [-0.99899259, -2.18185028, -0.15501118],
        [ 1.51546144, -2.51990523, -2.4978476 ]],

       [[ 2.32143909, -0.88700755,  3.53627377],
        [ 1.20273443,  1.66461369, -1.48155145],
        [ 0.48679434, -0.59104926, -1.3739001 ],
        [ 2.1670538 ,  1.31817276,  2.30587337],
        [ 2.87813867,  1.35685337,  0.49990663]],

       [[-0.05492616, -1.30325044, -0.23632161],
        [ 1.28126445,  0.55159012, -0.85967946],
        [-0.58622546,  1.6697768 , -0.74231627],
        [ 0.7499729 ,  1.49392337,  1.15262208],
        [ 1.2451601 ,  0.01887588, -3.99485055]],

       [[ 0.93858834,  0.53590005,  0.62824235],
        [-0.76116868, -0.60160413,  0.79169276],
        [ 0.25205217, -2.11996125, -5.98356696],
        [ 0.01833364,  0.4414261 ,  2.18705408],
        [-0.23230139, -0.25590461, -0.30662916]]])

In [36]:
D = 3
prior = GaussianPrior(D=D)
simulator = GaussianSimulator(D=D, s = np.array([1,2,3]))

theta = prior(4)
print(simulator.sigma)
simulator(theta, 5)

[[1. 0. 0.]
 [0. 2. 0.]
 [0. 0. 3.]]


array([[[ 0.36473194,  2.24128919, -2.35089873],
        [ 1.42798558,  0.35463437, -4.69827442],
        [ 0.69915346,  3.58725298, -0.80435372],
        [-0.6698736 ,  3.0819498 , -3.60022187],
        [ 0.31658728,  1.30124531, -4.66249927]],

       [[-0.06204786, -0.9222357 ,  4.09978518],
        [ 0.39526978,  1.31809759,  0.16983252],
        [-0.50249584,  1.89365678, -0.8629385 ],
        [ 0.61543453,  1.61838707,  3.82019019],
        [-0.96034709,  0.69172513, -2.01143108]],

       [[ 3.63896251,  0.48983212, -1.85047281],
        [ 1.04538554, -0.07861938,  0.90908639],
        [ 3.26866086, -0.99033381, -1.87555097],
        [ 2.06104198,  4.07258058,  2.29137259],
        [ 3.14722374, -2.14132504, -0.26769181]],

       [[ 1.13061978, -2.0111402 ,  2.30362794],
        [-0.8841437 , -2.37375824, -0.17239862],
        [-0.59448548, -0.22762924,  0.71078535],
        [-0.5551765 ,  0.34742613, -2.13734165],
        [-0.18849833, -1.55358033, -0.07487211]]])

In [49]:
D = 3
prior = GaussianPrior(D=D)
simulator = GaussianSimulator(D=D, s = np.array([[1,0,0],[0,1,0],[0,0,1]]))

theta = prior(4)
print(simulator.sigma)
simulator(theta, 5)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


array([[[-0.07887462,  1.50083976, -0.90590689],
        [-1.89543726,  0.56217446,  0.69306775],
        [ 0.47515784, -0.43629369,  1.22786432],
        [ 1.2265235 , -1.37148871,  0.93586156],
        [ 0.42180188,  0.66131824, -0.84900653]],

       [[ 0.11737355,  0.26568079,  0.78357155],
        [-2.09838154, -1.33648554,  0.36211326],
        [ 0.19691241,  0.24003328,  0.99406279],
        [-2.00595129, -1.20786454,  0.88785813],
        [-1.39010065,  0.81320097, -1.25135177]],

       [[ 1.99616164, -0.89792513, -0.36396683],
        [ 3.55504484, -0.09699204, -0.915926  ],
        [ 1.35228848, -0.03318188, -0.25753962],
        [ 2.12007553,  0.48199856,  0.57664428],
        [ 0.40502411, -2.46843758, -0.36071661]],

       [[ 0.57486689,  1.23464735, -0.17435137],
        [-1.95624394,  0.20516485, -0.72155552],
        [-0.24072445, -0.15743202, -2.21825009],
        [ 0.64383408,  1.70449472,  0.15044481],
        [-2.29020063, -1.82333524,  0.55604818]]])

In [10]:
D = 5 

#########

prior = GaussianPrior(D=D)
simulator = GaussianSimulator(D=D)
generative_model = GenerativeModel(prior, simulator)

#########

summary_meta = {
    'n_dense_s1': 2,
    'n_dense_s2': 2,
    'n_dense_s3': 2,
    'n_equiv':    1,
    'dense_s1_args': {'activation': 'relu', 'units': 32},
    'dense_s2_args': {'activation': 'relu', 'units': 32},
    'dense_s3_args': {'activation': 'relu', 'units': 32},
}

class BottleneckSummaryNet(tf.keras.Model):
    def __init__(self, inv_meta={}, n_out=10, activation_out=None):
        super(BottleneckSummaryNet, self).__init__()

        self.invariant_net = InvariantNetwork(inv_meta)
        self.out_layer = tf.keras.layers.Dense(n_out, activation=activation_out)
    
    def __call__(self, x):
        out_inv = self.invariant_net(x)
        out = self.out_layer(out_inv)
        return out


summary_net = BottleneckSummaryNet(inv_meta=summary_meta, 
                                   n_out=D*2,  # one mean and variance per dim
                                   activation_out=None  # linear
                                  )


inference_meta = {
    'n_coupling_layers': 2,
    's_args': {
        'units': [32, 32, 32],
        'activation': 'elu',
        'initializer': 'glorot_uniform',
    },
    't_args': {
        'units': [32, 32, 32],
        'activation': 'elu',
        'initializer': 'glorot_uniform',
    },
    'n_params': D,
    'alpha': 1.9,
    'permute': True
}

inference_net = InvertibleNetwork({'n_params': D})

amortizer = SingleModelAmortizer(inference_net, summary_net)

In [ ]:
trainer = ParameterEstimationTrainer(amortizer,
                      generative_model,
                      learning_rate=0.0001,
                      checkpoint_path='ckpt/'
                     )

In [ ]:
losses = trainer.train_online(epochs=1, iterations_per_epoch=100, batch_size=512, n_obs=100)

In [ ]:
losses = trainer.train_rounds(epochs=10, rounds=5, sim_per_round=20000, batch_size=512, n_obs=100)

In [ ]:
# Validate (quick and dirty)
p, x = trainer._forward_inference(200, 100)
param_samples = trainer.network.sample(x, n_samples=200)
param_means = param_samples.mean(axis=0)
true_vs_estimated(p, param_means, ['mu{}'.format(i) for i in range(1, 5+1)], figsize=(20,4))

In [ ]:
trainer.network.summary_net(x).shape

In [ ]:
trainer.manager.save()